In [9]:
import os

os.environ["ACADOS_SOURCE_DIR"] = "/root/robotic-mpc/acados"
os.environ["LD_LIBRARY_PATH"] = "/root/robotic-mpc/acados/lib:" + os.environ.get("LD_LIBRARY_PATH", "")
print("ACADOS_SOURCE_DIR =", os.environ["ACADOS_SOURCE_DIR"])
print("LD_LIBRARY_PATH   =", os.environ["LD_LIBRARY_PATH"])


ACADOS_SOURCE_DIR = /root/robotic-mpc/acados
LD_LIBRARY_PATH   = /root/robotic-mpc/acados/lib:


In [10]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[0]  # o [1] se serve
sys.path.append(str(PROJECT_ROOT))

from Experiments.simulator import Simulator, SimulationManager
import numpy as np

BASE_SURFACE_CONFIG = np.array({'a': -0.1, 'b': 0.1, 'c': -0.01, 'd': 0.01, 'e': 0.01, 'f': 0.0})
# Define joint limits for plotting boundaries
JOINT_POSITION_LIMITS = np.array([np.pi, np.pi, np.pi, np.pi, np.pi, np.pi])
JOINT_VELOCITY_LIMITS = np.array([3.15, 3.15, 3.15, 3.15, 3.15, 3.15])
JOINT_ACCELERATION_LIMITS = np.array([10.0, 10.0, 10.0, 10.0, 10.0, 10.0])

CROSS_OVER_FREQUENCIES = np.array([228.9, 262.09, 517.3, 747.44, 429.9, 1547.76]) # Simone's Magic Formula
SAMPLING_TIME = np.pi / (5*np.max(1/CROSS_OVER_FREQUENCIES)) #Shannon's theorem

base_config = {
    'dt': 0.0005,
    'simulation_time': 8,
    'surface_limits': ((-2, 2), (-2, 2)),
    'surface_origin': np.array([0.0, 0.0, 0.0]),
    'surface_orientation_rpy': np.array([0.0, 0.0, 0.0]),
    #surface function coefficients for a paraboloid
    'qdot_0': np.array([0, 0, 0, 0, 0, 0]),
    'q_0': np.array([np.pi/3, -np.pi/3, np.pi/4, -np.pi/2, -np.pi/2, 0.0]),
    'wcv': np.array([228.9, 262.09, 517.3, 747.44, 429.9, 1547.76]),
    'q_min': np.array([-2*np.pi, -2*np.pi, -np.pi, -2*np.pi, -2*np.pi, -2*np.pi], dtype=float),
    'q_max': np.array([+2*np.pi, +2*np.pi, +np.pi, +2*np.pi, +2*np.pi, +2*np.pi], dtype=float),
    'qdot_min': np.array([-np.pi, -np.pi, -np.pi, -np.pi, -np.pi, -np.pi], dtype=float),
    'qdot_max': np.array([np.pi, np.pi, np.pi, np.pi, np.pi, np.pi], dtype=float),
    'scene': True,
    'prediction_horizon': 200,  # Default value
    'solver_options': { 
        'nlp_solver_type': 'SQP',
        'qp_solver': 'PARTIAL_CONDENSING_HPIPM'
    }
}
manager = SimulationManager(base_config)

In [11]:
# Extract the dictionary from the numpy array
base_dict = BASE_SURFACE_CONFIG.item()

# Generate 10 varied surface configurations
np.random.seed(42)  # For reproducibility, remove if you want different sim_results each time
n_configs = 3
std = 0.01

surface_configs = []
for i in range(n_configs):
    config = {}
    for key, mean in base_dict.items():
        # Sample from normal distribution with mean from base config and std of 1
        config[key] = np.random.normal(mean, std)
    surface_configs.append(config)

# Print the generated configs to verify
print(f"Generated {len(surface_configs)} surface configurations:")
for i, config in enumerate(surface_configs):
    print(f"Config {i}: {config}")



Generated 3 surface configurations:
Config 0: {'a': -0.09503285846988768, 'b': 0.09861735698828816, 'c': -0.0035231146189930755, 'd': 0.025230298564080253, 'e': 0.00765846625276664, 'f': -0.0023413695694918055}
Config 1: {'a': -0.08420787184492609, 'b': 0.1076743472915291, 'c': -0.014694743859349522, 'd': 0.015425600435859648, 'e': 0.005365823071875378, 'f': -0.004657297535702569}
Config 2: {'a': -0.09758037728433966, 'b': 0.08086719755342203, 'c': -0.02724917832513033, 'd': 0.004377124707590273, 'e': -0.00012831120334423803, 'f': 0.003142473325952739}


In [12]:
# Run grid search with the varied configurations
manager.clear()
prediction_horizons = [5, 10, 20, 50, 100, 200]
manager.grid_search(

    param_grid={
        'prediction_horizon': prediction_horizons
    },
    surface_coeff_sets=surface_configs,
    name_template=lambda p, c: f"surf{c}_H{p['prediction_horizon']}"
)

sim_results = manager.run_all()


[1/18] Running: surf0_H5
URDF successfully loaded: /root/robotic-mpc/Infrastructure/ur_description/urdf/ur5.urdf
nq = 6, ngeoms(vis) = 7, ngeoms(col) = 7
Pinocchio model and data successfully created.
qp_solver = PARTIAL_CONDENSING_HPIPM
nlp_solver_type = SQP
rm -f libacados_ocp_solver_six_dof_robot_79d797f9.so
rm -f six_dof_robot_79d797f9_cost/six_dof_robot_79d797f9_cost_y_0_fun.o six_dof_robot_79d797f9_cost/six_dof_robot_79d797f9_cost_y_0_fun_jac_ut_xt.o six_dof_robot_79d797f9_cost/six_dof_robot_79d797f9_cost_y_0_hess.o six_dof_robot_79d797f9_cost/six_dof_robot_79d797f9_cost_y_fun.o six_dof_robot_79d797f9_cost/six_dof_robot_79d797f9_cost_y_fun_jac_ut_xt.o six_dof_robot_79d797f9_cost/six_dof_robot_79d797f9_cost_y_hess.o acados_solver_six_dof_robot_79d797f9.o
cc -fPIC -std=c99   -O2 -I/root/robotic-mpc/acados/include -I/root/robotic-mpc/acados/include/acados -I/root/robotic-mpc/acados/include/blasfeo/include -I/root/robotic-mpc/acados/include/hpipm/include  -c -o six_dof_robot_79d797f

OSError: libqpOASES_e.so: cannot open shared object file: No such file or directory

In [ ]:
from Reporting.plotter import Plotter
plotter = Plotter()

# Select specific simulations for detailed plots
selected_sim_results = [r for r in sim_results if r['name'] in [ 'surf0_H50', 'surf0_H200']]
# Get time vector from first selected result
t = selected_sim_results[0]['data']['time']
i = np.arange(selected_sim_results[0]['simulator'].Nsim)

box_data = {}
for r in sim_results:
    group_label = str(r['simulator'].prediction_horizon)
    if group_label not in box_data:
        box_data[group_label] = []
    box_data[group_label].append(r['summary']['rmse_e1'])

fig_box = plotter.box_plot(
    data=box_data,
    xlabel="Prediction Horizon",
    ylabel="RMSE e1 [m]",
    title="Surface Tracking Error Distribution vs Prediction Horizon"
)

# Error tracking plots
fig_e1 = plotter.generic_plot(
    t, 
    *[r['analysis']['e1'] for r in selected_sim_results],
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$e_1 \\ [\\text{m}]$", 
    title="Constraint 1 — Surface contact: $e_1 = S(p_x^{\\text{task}}, p_y^{\\text{task}}) - p_z^{\\text{task}}$", 
    labels=[r['name'] for r in selected_sim_results]
)

fig_e2 = plotter.generic_plot(
    t, 
    *[r['analysis']['e2'] for r in selected_sim_results],
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$e_2$", 
    title="Constraint 2 — Normal alignment: $e_2 = \\frac{\\mathbf{n}^{\\mathsf{T}}}{\\lVert \\mathbf{n} \\rVert} \\mathbf{r}_z - 1$", 
    labels=[r['name'] for r in selected_sim_results]
)

fig_e3 = plotter.generic_plot(
    t, 
    *[r['analysis']['e3'] for r in selected_sim_results],
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$e_3$", 
    title="Constraint 3 — Lateral axis orthogonality: $e_3 = \\begin{bmatrix} 1 & 0 & 0 \\end{bmatrix} \\mathbf{r}_y - 0$", 
    labels=[r['name'] for r in selected_sim_results]
)

fig_e4 = plotter.generic_plot(
    t, 
    *[r['analysis']['e4'] for r in selected_sim_results],
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$e_4 \\ [\\text{m}]$", 
    title="Constraint 4 — Fixed $x$ position: $e_4 = p_x^{\\text{task}} - p_{x,\\mathrm{ref}}$", 
    labels=[r['name'] for r in selected_sim_results]
)

fig_e5 = plotter.generic_plot(
    t, 
    *[r['analysis']['e5'] for r in selected_sim_results],
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$e_5 \\ [\\text{m/s}]$", 
    title="Constraint 5 — Tangential velocity: $e_5 = v^{\\text{task}}_{y} - v_{y,\\mathrm{ref}}$", 
    labels=[r['name'] for r in selected_sim_results]
)

# Control Input Comparison - Joint 1
fig_u1 = plotter.generic_plot(
    t, 
    *[r['data']['u'][0] for r in selected_sim_results],
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$u_1 \\ [\\text{rad/s}]$", 
    title="Control Input - Joint 1", 
    labels=[r['name'] for r in selected_sim_results]
)

    # Timing Performance
fig_mpc_time = plotter.generic_plot(
    t, 
    *[r['analysis']['mpc_time'] for r in selected_sim_results],
    upper_bound=selected_sim_results[0]['simulator'].dt,
    xlabel="$t \\ [\\text{s}]$", 
    ylabel="$\\text{MPC Time} \\ [\\text{s}]$", 
    title=f"MPC Computation Time (dt = {selected_sim_results[0]['simulator'].dt}s)", 
    labels=[r['name'] for r in selected_sim_results]
)

rmse_data = {
    "$e_1$ (surface)": [r['summary']['rmse_e1'] for r in sim_results],
    "$e_2$ (x-pos)": [r['summary']['rmse_e2'] for r in sim_results],
    "$e_3$ (orient)": [r['summary']['rmse_e3'] for r in sim_results],
    "$e_4$ (y-vel)": [r['summary']['rmse_e4'] for r in sim_results],
}
fig_rmse = plotter.grouped_bar_plot(
    data=rmse_data,
    group_labels=[r['name'] for r in sim_results],
    xlabel="Simulation",
    ylabel="RMSE",
    title="Constraint RMSE Comparison"
)

# Grouped bar plot for ITSE comparison
itse_data = {
    "$e_1$ (surface)": [r['summary']['itse_e1'] for r in sim_results],
    "$e_2$ (x-pos)": [r['summary']['itse_e2'] for r in sim_results],
    "$e_3$ (orient)": [r['summary']['itse_e3'] for r in sim_results],
    "$e_4$ (y-vel)": [r['summary']['itse_e4'] for r in sim_results],
}
fig_itse = plotter.grouped_bar_plot(
    data=itse_data,
    group_labels=[r['name'] for r in sim_results],
    xlabel="Simulation",
    ylabel="ITSE",
    title="Constraint ITSE Comparison"
)

# Bar chart for total computation time
fig_total_time = plotter.bar_plot(
    values=[r['summary']['total_computation_time'] for r in sim_results],
    labels=[r['name'] for r in sim_results],
    xlabel="Simulation",
    ylabel="Total Time [s]",
    title="Total Computation Time (MPC + Integration)"
)


display(fig_e1, fig_e2, fig_e3, fig_e4, fig_e5, fig_u1)
display(fig_rmse, fig_itse, fig_total_time, fig_mpc_time)

display(fig_box)


In [59]:
fig_q = plotter.joints(t, *[r['data']['q'] for r in selected_sim_results], labels=[r['name'] for r in selected_sim_results], lower_bounds=JOINT_POSITION_LIMITS, upper_bounds=JOINT_POSITION_LIMITS)
fig_qdot = plotter.joints(t, *[r['data']['qdot'] for r in selected_sim_results], labels=[r['name'] for r in selected_sim_results], lower_bounds=JOINT_VELOCITY_LIMITS, upper_bounds=JOINT_VELOCITY_LIMITS)
display(fig_q, fig_qdot)

In [58]:

res = sim_results[0]['analysis']
fig_res_components = plotter.generic_plot(
    t,
    res['res_stat'],
    res['res_eq'],
    res['res_ineq'],
    res['res_comp'],
    ylog=True,
    xlabel="$t$ [s]",
    ylabel="Residual",
    title=f"Residual Components - {sim_results[0]['name']}",
    labels=["Stationarity", "Equality", "Inequality", "Complementarity"]
)

# SQP Iterations
fig_sqp = plotter.generic_plot(
    t,
    *[r['analysis']['sqp_iterations'] for r in selected_sim_results],
    xlabel="$t$ [s]",
    ylabel="SQP Iterations",
    title="SQP Iterations per MPC Solve",
    labels=[r['name'] for r in selected_sim_results]
)

# Cost history - Should be final cost at each iteration.
fig_cost = plotter.generic_plot(
    i ,
    *[r['analysis']['cost_history'] for r in selected_sim_results],
    ylog=True,
    xlabel="$i$ [s]",
    ylabel="Cost",
    title="Final Cost at each simulation step",
    labels=[r['name'] for r in selected_sim_results]
)



display(fig_res_components, fig_sqp, fig_cost, fig_mpc_time)

In [61]:
plotter.gen_html_report(
    task_figs=[fig_e1, fig_e2, fig_e3, fig_e4, fig_e5, fig_u1, fig_q, fig_qdot, fig_q, fig_qdot],
    solver_figs=[fig_res_components, fig_sqp, fig_cost, fig_mpc_time],
    video_folder=None,
    title="Surface Tracking Analysis",
    filename="surface_tracking_analysis.html"
)